# **Automating Data Analysis with PythonREPLTool and LLMs**

## **Introduction**

In today’s data-driven world, businesses and individuals alike rely heavily on data analysis to make informed decisions. However, the process of analyzing data—loading datasets, performing calculations, and generating visualizations—can often be time-consuming and require specialized skills. Enter **large language models (LLMs)** like OpenAI’s GPT-4, which have revolutionized how we interact with technology. By combining the natural language understanding of LLMs with custom tools, we can automate complex workflows and make data analysis more accessible to everyone.

This example demonstrates how to use **LangChain**, a powerful framework for building LLM-powered applications, to create a conversational data analysis assistant. The assistant will handle a complete data analysis task: loading a CSV file containing sales data, calculating the total sales for each product, generating a bar plot to visualize the results, and providing a final summary—all through a simple conversational interface. By integrating LangChain with custom Python tools, we can bridge the gap between natural language and data analysis, enabling users to interact with data in a more intuitive and efficient way.

The workflow is designed to be flexible and scalable, making it suitable for a wide range of data analysis tasks. Whether you’re a business analyst looking to automate repetitive tasks or a developer exploring the potential of LLMs, this example provides a practical foundation for building intelligent, tool-enhanced conversational agents. Let’s dive into the step-by-step process of creating this assistant and see how it can transform the way we work with data.

---

## Preparation

### Installing Required Libraries
This section installs the necessary Python libraries for working with LangChain, OpenAI embeddings, Anthropic models, DuckDuckGo search, and other utilities. These libraries include:
- `langchain-openai`: Provides integration with OpenAI's embedding models and APIs.
- `langchain-anthropic`: Enables integration with Anthropic's models and APIs.
- `langchain_community`: Contains community-contributed modules and tools for LangChain.
- `langchain_experimental`: Includes experimental features and utilities for LangChain.
- `langgraph`: A library for building and visualizing graph-based workflows in LangChain.
- `duckduckgo-search`: Enables programmatic access to DuckDuckGo's search functionality.

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain_community
!pip install -qU langchain_experimental
!pip install -qU langgraph
!pip install -qU duckduckgo-search

### **Create a CSV File with Sales Data**
This block creates a CSV file named `sales_data.csv` containing sales data for three products over five days.

In [ ]:
import pandas as pd

# Define the data
data = {
    "Date": [
        "2023-10-01", "2023-10-01", "2023-10-01",
        "2023-10-02", "2023-10-02", "2023-10-02",
        "2023-10-03", "2023-10-03", "2023-10-03",
        "2023-10-04", "2023-10-04", "2023-10-04",
        "2023-10-05", "2023-10-05", "2023-10-05",
    ],
    "Product": [
        "Product A", "Product B", "Product C",
        "Product A", "Product B", "Product C",
        "Product A", "Product B", "Product C",
        "Product A", "Product B", "Product C",
        "Product A", "Product B", "Product C",
    ],
    "Sales": [
        1000, 500, 1500,
        1200, 600, 1800,
        900, 400, 1300,
        1100, 550, 1600,
        1300, 700, 2000,
    ],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("sales_data.csv", index=False)

print("sales_data.csv created successfully!")

## **Step 1: Define Custom Tools**
Define custom tools for data analysis using the `@tool` decorator. These tools will handle specific tasks like loading CSV files and generating plots.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
import pandas as pd
import matplotlib.pyplot as plt

@tool
def csv_loader(file_path: str) -> str:
    """Load a CSV file into a pandas DataFrame and return it as JSON."""
    return pd.read_csv(file_path).to_json(orient="split")  # Use "split" orientation for better compatibility

@tool
def plot(data_json: str, plot_type: str, x: str, y: str) -> str:
    """Generate a plot from a pandas DataFrame and save it as an image."""
    try:
        df = pd.read_json(data_json, orient="split")  # Use "split" orientation for better compatibility
        if plot_type == "line":
            df.plot(x=x, y=y, kind="line")
        elif plot_type == "bar":
            df.plot(x=x, y=y, kind="bar")
        plt.savefig("plot.png")
        return "Plot saved as plot.png"
    except Exception as e:
        return f"Error generating plot: {str(e)}"

## **Step 2: Initialize the LLM**
Initialize the `ChatOpenAI` model with the `gpt-4o-mini` model and set the temperature to `0` for deterministic responses. Fetch the API key securely using `kaggle_secrets`.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=user_secrets.get_secret("my-openai-api-key"))
#model = ChatAnthropic(model="claude-3-5-latest", temperature=0, api_key=user_secrets.get_secret("my-anthropic-api-key"))

## **Step 3: Initialize the Tools**
Initialize the tools, including the built-in `PythonREPLTool` and the custom tools (`csv_loader` and `plot`).

In [ ]:
from langchain_experimental.tools.python.tool import PythonREPLTool

python_tool = PythonREPLTool()
csv_loader_tool = csv_loader
plot_tool = plot

## **Step 4: Bind the Tools to the LLM**
Bind the tools to the `ChatOpenAI` model using the `bind_tools()` method. This allows the model to dynamically invoke these tools based on the user's request.

In [ ]:
model_with_tools = model.bind_tools([python_tool, csv_loader_tool, plot_tool])

## **Step 5: Define the User Prompt**
Define the user prompt, which requests the assistant to:
1. Load a CSV file.
2. Calculate total sales for each product.
3. Generate a bar plot of total sales by product.
4. Display the total sales and the plot.

In [ ]:
user_prompt = """
I have a CSV file named 'sales_data.csv' with columns: 'Date', 'Product', 'Sales'.
1. Load the CSV file.
2. Calculate the total sales for each product.
3. Generate a bar plot of total sales by product.
4. Tell me the total sales for each product and show the plot.
"""

## **Step 6: Create a Conversation History**
Create a conversation history with a `SystemMessage` to set the assistant's role and a `HumanMessage` containing the user's prompt.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

messages = [
    SystemMessage(content="You are a helpful data analysis assistant."),
    HumanMessage(content=user_prompt),
]

## **Step 7: Process Tool Calls Iteratively**
Process tool calls iteratively until the LLM generates a final response. This involves:
1. Sending the conversation history to the LLM.
2. Checking for tool calls in the response.
3. Invoking the appropriate tool and updating the conversation history.
4. Breaking the loop when no more tool calls are generated.

In [ ]:
max_iterations = 5  # Prevent infinite loops
for _ in range(max_iterations):
    response = model_with_tools.invoke(messages)

    # Check if the response contains tool calls
    if not response.additional_kwargs.get("tool_calls"):
        # No more tool calls, break the loop
        break

    # Process tool calls
    print("Processing tool calls...")  # Debug message
    for tool_call in response.additional_kwargs.get("tool_calls", []):
        tool_name = tool_call["function"]["name"]
        tool_args = eval(tool_call["function"]["arguments"])

        if tool_name == "csv_loader":
            # Load the CSV file
            tool_result = csv_loader_tool.invoke(tool_args["file_path"])
            messages.append(AIMessage(content="", additional_kwargs={"tool_calls": [tool_call]}))
            messages.append(ToolMessage(content=tool_result, tool_call_id=tool_call["id"]))

        elif tool_name == "Python_REPL":
            # Execute Python code (e.g., calculate total sales)
            tool_result = python_tool.run(tool_args["query"])
            messages.append(AIMessage(content="", additional_kwargs={"tool_calls": [tool_call]}))
            messages.append(ToolMessage(content=tool_result, tool_call_id=tool_call["id"]))

        elif tool_name == "plot":
            # Generate a plot
            tool_result = plot_tool.invoke(tool_args)  # Pass tool_args as input
            messages.append(AIMessage(content="", additional_kwargs={"tool_calls": [tool_call]}))
            messages.append(ToolMessage(content=tool_result, tool_call_id=tool_call["id"]))

## **Step 8: Print the Final Response**
Print the final response from the LLM, which includes the total sales for each product and a confirmation that the plot has been saved.

In [ ]:
print("Final Answer:\n", response.content)

---

## **Conclusion**

This example highlights the transformative potential of integrating large language models with custom tools to automate and simplify data analysis tasks. By leveraging LangChain, we’ve built a conversational assistant that can dynamically load data, perform calculations, and generate visualizations—all through natural language interactions. The workflow is not only efficient but also highly adaptable, allowing it to be extended to more complex tasks or integrated into larger systems.

The ability to interact with data using natural language opens up new possibilities for accessibility and productivity. Users no longer need to write complex code or navigate specialized software to analyze data; instead, they can simply describe their needs in plain English and let the assistant handle the rest. As LLMs continue to evolve, their integration with domain-specific tools will unlock even more opportunities for innovation, making advanced data analysis capabilities available to a broader audience.

Whether you’re analyzing sales data, visualizing trends, or automating repetitive tasks, this approach provides a powerful and user-friendly way to interact with data. By combining the strengths of LLMs and custom tools, we can create intelligent systems that empower users to make data-driven decisions with ease. The future of data analysis is conversational, and this example is just the beginning.